In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import RandomLinkSplit

In [16]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, 2 * out_channels, cached=True) # Eventually change layer type
        self.conv2 = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True) # Eventually change layer type

    def forward(self, x, edge_index): # Modify to adapt to the dataset complexity
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
    
class Decoder(nn.Module):
    def __init__(self, in_channels):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_channels, 1)

    def forward(self, z, edge_index):
        # By default, it computes the inner product of the node embeddings
        adj = torch.matmul(z[edge_index[0]], z[edge_index[1]].t())
        return adj[edge_index[0], edge_index[1]]
    
# Defining traing and evaluation functions
def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_data.edge_index.to(device))
    loss = model.recon_loss(z, train_data.pos_edge_label_index.to(device))
    loss.backward()
    optimizer.step()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    # Encode the node features to get the embeddings
    with torch.no_grad():
        z = model.encode(x, train_data.edge_index.to(device))
    
    # Compute the reconstruction loss
    return model.test(z, pos_edge_index.to(device), neg_edge_index.to(device))
    

In [ ]:
# Model definition
import os
LATENT_DIM = 32
data_path = os.path.join('..', 'data', 'graph_data.pt')
dataset = torch.load(data_path, weights_only=False)
data = dataset['data']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Instantiate the customized encoder in GAE model
transform = RandomLinkSplit(is_undirected=True, split_labels=True,add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

model = pyg_nn.GAE(Encoder(data.num_features, LATENT_DIM)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = train_data.x.to(device)

Using device: cuda


In [18]:
# Training loop
for epoch in range(1, 401):
    loss = train(epoch)
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)

    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 010, AUC: 0.7734, AP: 0.7989
Epoch: 020, AUC: 0.7638, AP: 0.7920
Epoch: 030, AUC: 0.7721, AP: 0.7980
Epoch: 040, AUC: 0.7863, AP: 0.8083
Epoch: 050, AUC: 0.7932, AP: 0.8135
Epoch: 060, AUC: 0.8025, AP: 0.8205
Epoch: 070, AUC: 0.8093, AP: 0.8253
Epoch: 080, AUC: 0.8157, AP: 0.8293
Epoch: 090, AUC: 0.8219, AP: 0.8321
Epoch: 100, AUC: 0.8260, AP: 0.8332
Epoch: 110, AUC: 0.8249, AP: 0.8317
Epoch: 120, AUC: 0.8197, AP: 0.8283
Epoch: 130, AUC: 0.8143, AP: 0.8249
Epoch: 140, AUC: 0.8087, AP: 0.8213
Epoch: 150, AUC: 0.8024, AP: 0.8170
Epoch: 160, AUC: 0.8023, AP: 0.8171
Epoch: 170, AUC: 0.8005, AP: 0.8159
Epoch: 180, AUC: 0.8028, AP: 0.8176
Epoch: 190, AUC: 0.8002, AP: 0.8158
Epoch: 200, AUC: 0.8008, AP: 0.8163
Epoch: 210, AUC: 0.8011, AP: 0.8164
Epoch: 220, AUC: 0.8013, AP: 0.8166
Epoch: 230, AUC: 0.7978, AP: 0.8142
Epoch: 240, AUC: 0.7994, AP: 0.8154
Epoch: 250, AUC: 0.8020, AP: 0.8172
Epoch: 260, AUC: 0.7997, AP: 0.8156
Epoch: 270, AUC: 0.7985, AP: 0.8148
Epoch: 280, AUC: 0.7969, AP:

In [20]:
model.eval()
z = model.encode(x, train_data.edge_index.to(device))
emb_sample = z[0]
print(f'Number of nodes: {z.shape[0]}')
print(f'Number of edges: {train_data.edge_index.shape[1]}')
print(f'Embedding sample: {z}')

Number of nodes: 4877
Number of edges: 20618
Embedding sample: tensor([[ 1.4475],
        [ 1.1396],
        [ 0.6645],
        ...,
        [-0.7973],
        [-0.7973],
        [-0.7973]], device='cuda:0', grad_fn=<AddBackward0>)
